In [1]:
%run startup.py

Loading metadata history from /Users/dallan/.config/bluesky/bluesky_history.db


In [83]:
import time
from ophyd import Device, Signal, Component as Cpt, DeviceStatus
from ophyd.sim import SynSignal, SynPeriodicSignal

## Interface to Signal

In [54]:
sig = Signal(name='sig', value=3)
sig

Signal(name='sig', value=3, timestamp=1509712457.234915)

### Methods that require no communication with the IOC

In [18]:
sig.name

'sig'

In [20]:
sig.parent is None

True

### Methods that ask the IOC to tell us something it already 'knows'

In [21]:
sig.connected

True

In [22]:
sig.limits

(0, 0)

In [4]:
sig.read()

{'sig': {'timestamp': 1509711385.3563771, 'value': 3}}

In [25]:
sig.describe()

{'sig': {'dtype': 'number', 'shape': [], 'source': 'SIM:sig'}}

### Monitoring (subscribing for updates asynchronously)

In [114]:
def cb(value, old_value, **a_whole_bunch_of_junk):
    print(f'changed from {old_value} to {value}')
    
sig.subscribe(cb)
# The act of subscribing always generates one reading immediately...

changed from 3 to 5


2

If this were an `EpicsSignal` instead of a `Signal`, `cb` would be called from a thread every time pyepics receives a new update about the value of `sig`. In this case, we have to update it manually.

In [107]:
sig.put(5)

changed from 3 to 5
changed from 3 to 5


In [ ]:
sig.put(10)

Or we can use `SynPeriodicSignal`, which updates at randomized intervals.

In [115]:
from ophyd.sim import SynPeriodicSignal

rand = SynPeriodicSignal(name='rand')
token = rand.subscribe(cb)

changed from 0.8688308772990598 to 0.26846871475299205
changed from 0.26846871475299205 to 0.11261979130702737
changed from 0.11261979130702737 to 0.7432412503829011
changed from 0.7432412503829011 to 0.48295350158473205
changed from 0.48295350158473205 to 0.6678032232275664


In [117]:
rand.unsubscribe(token)

### Methods that ask the IOC to take a (potentially lengthy) action

In [77]:
def cb():
    print("finished at t =", time.time())

status = sig.set(5)
status.add_callback(cb)

finished at t = 1509712809.176476


In [40]:
status

Status(obj=Signal(name='sig', value=5, timestamp=1509712182.923359), done=True, success=True)

In [41]:
status.done

True

In [42]:
status = sig.trigger()
status.add_callback(cb)

finished at t = 1509712207.3577878


## Interface of a Status object

In [86]:
status = DeviceStatus(sig)

In [87]:
status.done

False

In [88]:
status.success

False

In [91]:
def cb():
    print("BOOM")

status.add_callback(cb)

In [93]:
status.callbacks

deque([<function __main__.cb>])

In [95]:
status.device  # the Device or Signal that the Status pertains to

Signal(name='sig', value=5, timestamp=1509712809.1763132)

In [96]:
status._finished()

BOOM


In [97]:
status.done

True

In [98]:
status.success

True

In [99]:
# Failure looks like this:
status = DeviceStatus(sig)
status.add_callback(cb)
status._finished(success=False)
status.success

BOOM


False

We'll see later how to actually use this in practice.

## Interface to Device

In [63]:
# This encodes the _structure_ of a kind of Device.
# Real examples include EpicsMotor, EpicsScaler or user-defined
# combinations of these, such as a platform that can move in X and Y.

class Platform(Device):
    x = Cpt(Signal, value=3)
    y = Cpt(Signal, value=4)
    
p1 = Platform(name='p1')
p2 = Platform(name='p2')

### Names and relationships

In [64]:
p1

Platform(prefix='', name='p1', read_attrs=['x', 'y'], configuration_attrs=[])

In [72]:
p1.component_names

['x', 'y']

In [65]:
p1.x

Signal(name='p1_x', parent='p1', value=3, timestamp=1509712696.649813)

In [73]:
p1.y

Signal(name='p1_y', parent='p1', value=4, timestamp=1509712696.649896)

In [66]:
p1.name

'p1'

In [67]:
p1.x.name

'p1_x'

In [68]:
p1.x.parent is p1

True

### Reading the parent combines the readings of its children

In [74]:
p1.read()

OrderedDict([('p1_x', {'timestamp': 1509712696.649813, 'value': 3}),
             ('p1_y', {'timestamp': 1509712696.649896, 'value': 4})])

In [70]:
p1.x.read()

{'p1_x': {'timestamp': 1509712696.649813, 'value': 3}}

and `describe` works exactly the same way:

In [75]:
p1.describe()

OrderedDict([('p1_x', {'dtype': 'number', 'shape': [], 'source': 'SIM:p1_x'}),
             ('p1_y', {'dtype': 'number', 'shape': [], 'source': 'SIM:p1_y'})])

In [76]:
p1.x.describe()

{'p1_x': {'dtype': 'number', 'shape': [], 'source': 'SIM:p1_x'}}

### Components are sorted into categories:

* `read_attrs` -- things to read once per Event (i.e. row in the table)
* `configuration_attrs` -- things to read once per Event Descriptor (which usually means one per run)
* things ommitted from data collection entirely, but available for debugging etc.

In [118]:
p1.read_attrs

['x', 'y']

In [119]:
p1.configuration_attrs

[]

In [137]:
# dumb example...

class Platform(Device):
    _default_configuration_attrs = ('motion_compensation',)
    _default_read_attrs = ('x', 'y')
    x = Cpt(Signal, value=3)
    y = Cpt(Signal, value=4)
    motion_compensation = Cpt(Signal, value=1)  # a boolean
    
p1 = Platform(name='p1')
p2 = Platform(name='p2')

In [139]:
p1.read_attrs

['x', 'y']

In [138]:
p1.configuration_attrs

['motion_compensation']

In [136]:
p1.read_configuration()

OrderedDict([('p1_motion_compensation',
              {'timestamp': 1509715147.724662, 'value': 1})])

In [141]:
p1.describe_configuration()

OrderedDict([('p1_motion_compensation',
              {'dtype': 'number',
               'shape': [],
               'source': 'SIM:p1_motion_compensation'})])

The data from `configuration_attrs` isn't displayed by the built-in callbacks...

In [143]:
RE(count([p1]))

Transient Scan ID: 42     Time: 2017/11/03 09:22:47
Persistent Unique Scan ID: 'c14bc80e-48af-48c2-ba93-ad932e4a2b5a'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |       p1_x |       p1_y |
+-----------+------------+------------+------------+
|         1 | 09:22:47.8 |      3.000 |      4.000 |
+-----------+------------+------------+------------+
generator count ['c14bc8'] (scan num: 42)





('c14bc80e-48af-48c2-ba93-ad932e4a2b5a',)

... but the data is saved, and it can accessed conveniently like so:

In [145]:
h = db[-1]
h.config_data('p1')

{'primary': [{'p1_motion_compensation': 1}]}

### 'Staging' -- a hook for putting a device into a controlled state for data collection (and then putting it back)

In [148]:
class Platform(Device):
    _default_configuration_attrs = ('motion_compensation',)
    _default_read_attrs = ('x', 'y')
    x = Cpt(Signal, value=3)
    y = Cpt(Signal, value=4)
    motion_compensation = Cpt(Signal, value=1)  # a boolean
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.stage_sigs['motion_compensation'] = 1
        

p1 = Platform(name='p1')

In [150]:
p1.motion_compensation.get()

1

In [151]:
p1.motion_compensation.put(0)

`Device.stage()` stashes the current state of the signals in `stage_sigs` and then puts the device into the desired state.

In [152]:
p1.stage()

[Platform(prefix='', name='p1', read_attrs=['x', 'y'], configuration_attrs=['motion_compensation'])]

In [153]:
p1.motion_compensation.get()

1

`Device.unstage()` uses that stashed stage to put everything back.

In [154]:
p1.unstage()

[Platform(prefix='', name='p1', read_attrs=['x', 'y'], configuration_attrs=['motion_compensation'])]

In [155]:
p1.motion_compensation.get()

0

Staging twice is illegal:

In [160]:
p1.stage()

[Platform(prefix='', name='p1', read_attrs=['x', 'y'], configuration_attrs=['motion_compensation'])]

In [161]:
p1.stage()

RedundantStaging: Device Platform(prefix='', name='p1', read_attrs=['x', 'y'], configuration_attrs=['motion_compensation']) is already staged. Unstage it first.

But unstaging is indempotent:

In [164]:
p1.unstage()
p1.unstage()
p1.unstage()

[Platform(prefix='', name='p1', read_attrs=['x', 'y'], configuration_attrs=['motion_compensation'])]

## Pseudopositioners

## Fly Scans!